In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [20]:
year = []
model = []
mileage = []
location = []
price = []
price_type = []
color_scheme = []
hist_cond = []

for i in range (1,31):
    
        # Website variable
        website = 'https://www.truecar.com/used-cars-for-sale/listings/bmw/3-series/year-2006-2011/location-effort-pa/?page=' + str(i) + '&searchRadius=5000&trimSlug[]=335i&trimSlug[]=335is&trimSlug[]=335xi'

        # Response to website
        response = requests.get(website)
        
        # Soup object
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Results
        results = soup.find_all('div', {'class':'linkable card card-shadow vehicle-card _1qd1muk'})
        
        for result in results:
            
            # year 
            try:
                year.append(result.find('span', {'class': 'vehicle-card-year font-size-1'}).get_text())
            except:
                year.append('n/a')
                
            # model
            try:
                model.append(result.find('div', {'class': 'font-size-1 text-truncate'}).get_text())
            except:
                model.append('n/a')
            
            # mileage
            try:
                mileage.append(result.find('div', {'data-test':'vehicleMileage'}).get_text())
            except:
                mileage.append('n/a')
            
            # location
            try:
                location.append(result.find('div', {'data-test':'vehicleCardLocation'}).get_text())
            except:
                location.append('n/a')
            
            # price
            try:
                price.append(result.find('div', {'data-test':'vehicleCardPricingBlockPrice'}).get_text())
            except:
                price.append('n/a')
            
            # price_type
            try:
                price_type.append(result.find('span', {'data-test':'graphIconLabel'}).get_text())
            except:
                price_type.append('n/a')
            
            # color_scheme
            try:
                color_scheme.append(result.find('div', {'data-test': 'vehicleCardColors'}).get_text())
            except:
                color_scheme.append('n/a')
            
            # hist_cond
            try:
                hist_cond.append(result.find('div', {'data-test':'vehicleCardCondition'}).get_text())
            except:
                hist_cond.append('n/a')

In [21]:
truecar_df = pd.DataFrame ({'Year':year, 'Model':model, 'Mileage':mileage,
                           'Location':location, 'Price':price, 'Site Price Type':price_type,
                           'Color Scheme':color_scheme, 'History':hist_cond})

In [22]:
truecar_df

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History
0,2011,335is Convertible,"62,515 miles","58 mi - Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use"
1,2011,335i xDrive Coupe AWD,"37,739 miles","21 mi - Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use"
2,2011,328i xDrive Sedan AWD SULEV,"80,434 miles","74 mi - South Hackensack, NJ","$9,795",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use"
3,2011,335i Sedan,"67,840 miles","8.7 mi - Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use"
4,2010,335i xDrive Sedan AWD,"128,507 miles","73 mi - Hasbrouck Heights, NJ","$9,995",Excellent Price,"Black exterior, Beige interior","No accidents, 3 Owners, Personal use"
...,...,...,...,...,...,...,...,...
925,2011,328i xDrive Sedan AWD SULEV,"119,730 miles","68 mi - Paterson, NJ","$11,486",High Price,"Black exterior, Black interior","No accidents, 1 Owner, Personal use"
926,2011,328i xDrive Sedan AWD,"129,862 miles","66 mi - Rahway, NJ","$9,900",High Price,"Unknown exterior, Unknown interior","1 accident, 4 Owners, Fleet use"
927,2011,335i Convertible,"89,479 miles","67 mi - Elizabeth, NJ","$18,985",High Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use"
928,2011,335i Sedan,"106,062 miles","72 mi - Garfield, NJ","$13,499",High Price,"Unknown exterior, Unknown interior","No accidents, 5 Owners, Personal use"


In [23]:
# Commented out so new file isn't made everytime script is executed.

# truecar_df.to_excel('uncleandata.xlsx', index=False)